In [1]:
import torch
import requests
from bs4 import BeautifulSoup
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoTokenizer, AutoModelForCausalLM
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
import ssl
import matplotlib.pyplot as plt
import pandas as pd
import os
from nltk.tokenize import TreebankWordTokenizer
import urllib3

In [2]:
# Configuram contextul SSL pentru a permite accesul la site-urile care au certificate nesigure
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

try:
    ssl._create_default_https_context = ssl._create_unverified_context
except AttributeError:
    pass
# Descarcam tokenizerul 'punkt' necesar pentru evaluarea BLEU
import nltk
nltk.download('punkt', quiet=True)
nltk.data.path.append("C:/Users/ungur/AppData/Roaming/nltk_data")
tokenizer_bleu = TreebankWordTokenizer()

In [3]:
# Incarcare sonete Shakespeare din web. Extrage paragrafele HTML si le converteste intr-o lista de liste de versuri
# Pastreaza doar acele paragrafe care contin cel putin 3 versuri

def incarca_sonete():
    raspuns = requests.get("https://www.shakespeares-sonnets.com/all.php", headers={'User-Agent': 'Mozilla/5.0'}, verify=False)
    soup = BeautifulSoup(raspuns.text, 'html.parser')
    paragrafe = soup.find_all('p')
    sonete = []
    for paragraf in paragrafe:
        text = paragraf.get_text(separator="\n").replace("\r", "").strip()
        linii = [l.strip() for l in text.split("\n") if l.strip()]
        if len(linii) >= 3:
            sonete.append(linii)
    return sonete

In [4]:
# Impartire poezii in strofe de 3 versuri

def extrage_strofe(poem):
    return [poem[i:i+3] for i in range(0, len(poem), 3) if len(poem[i:i+3]) == 3]


In [5]:
# Evaluare model pe baza scorului BLEU. Pentru fiecare strofa se ia primul vers (restul fiind "pierdut" dupa inundatie), 
# se genereaza 1-3 versuri noi, se compara versurile generate cu cele reale (care au fost pastrate pentru comparatie),
# se calculeaza scorul BLEU

def evalueaza_model(model, tokenizer, lista_strofe, lista_parametri, nume_model="GPT2", num_strofe=2):
    functie_smooth = SmoothingFunction().method4
    rezultate = []

    for i, (prim_vers, versuri_referinta) in enumerate(lista_strofe[:num_strofe]):
        for max_tokeni, temperatura, top_p in lista_parametri:
            # Pregatim input-ul pentru model
            intrare = tokenizer.encode(prim_vers + "\n", return_tensors="pt").to(model.device)

            # Generam continuarea strofei
            attention_mask = torch.ones_like(intrare)
            iesire = model.generate(
                intrare,
                max_length=intrare.shape[1] + max_tokeni,
                temperature=temperatura,
                top_p=top_p,
                do_sample=True,
                num_return_sequences=1,
                pad_token_id=tokenizer.eos_token_id,
                attention_mask=attention_mask
            )

            # Extragem textul generat si pastram doar primele 3 versuri
            text_generat = tokenizer.decode(iesire[0], skip_special_tokens=True)[len(prim_vers):].strip()
            linii_generate = [l for l in text_generat.split("\n") if l][:3]

            # Tokenizam atat referinta cat si generarea pentru calcul BLEU
            ref_tokeni = [tokenizer_bleu.tokenize(" ".join(versuri_referinta).lower())]
            cand_tokeni = tokenizer_bleu.tokenize(" ".join(linii_generate).lower())
            scor = sentence_bleu(ref_tokeni, cand_tokeni, smoothing_function=functie_smooth)

            # Afisam comparatia si scorul BLEU
            print(f"Scor BLEU: {scor:.4f}")

            rezultate.append({
                'model': nume_model,
                'prompt': prim_vers,
                'original': versuri_referinta,
                'generat': linii_generate,
                'bleu': scor,
                'max_tokeni': max_tokeni,
                'temperatura': temperatura,
                'top_p': top_p
            })
    return rezultate

In [6]:
# Testare prompturi in diferite limbi. Genereaza un text pe baza lui folosind modelul si tokenizerul specificat

def test_limba_prompt(model, tokenizer, prompt, descriere):
    print(f"\n Test {descriere}")
    intrare = tokenizer.encode(prompt + "\n", return_tensors="pt").to(model.device)
    iesire = model.generate(intrare, max_length=intrare.shape[1] + 60, temperature=0.7, top_p=0.9, do_sample=True)
    rezultat = tokenizer.decode(iesire[0], skip_special_tokens=True)[len(prompt):].strip()
    print(f"Prompt: {prompt}\nGenerat:\n{rezultat}")

In [7]:
# Genereaza o poezie cu stil pastel, pornind de la un vers de inceput. Adauga o indicatie stilistica ca hint textual pentru model

def genereaza_pastel(prompt, stil="pastoral scene with nature", max_tokeni=60):
    seed = prompt + f"\n[{stil}]"
    intrare = poet_tokenizer.encode(seed, return_tensors="pt").to(poet_model.device)
    iesire = poet_model.generate(intrare, max_length=intrare.shape[1]+max_tokeni, temperature=0.6, top_p=0.85, do_sample=True)
    rezultat = poet_tokenizer.decode(iesire[0], skip_special_tokens=True)[len(prompt):].strip()
    print(f"\n Stil pastel:\n{rezultat}")

In [8]:
# Incarca sonetele Shakespeare de pe web. Extrage strofe de 3 versuri. Incarca 2 modele: GPT2 (generalist) si unul poetic antrenat pe poezii
# Incarca tokenizatoarele corespunzatoare

def init_model_si_date():
    global gpt2_model, gpt2_tokenizer, poet_model, poet_tokenizer, strofe
    response = requests.get("https://www.shakespeares-sonnets.com/all.php", headers={'User-Agent': 'Mozilla/5.0'}, verify=False)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragrafe = soup.find_all('p')
    sonete = []
    for paragraf in paragrafe:
        text = paragraf.get_text(separator="\n").replace("\r", "").strip()
        linii = [l.strip() for l in text.split("\n") if l.strip()]
        if len(linii) >= 3:
            sonete.append(linii)
    strofe = [(s[0], s[1:]) for poem in sonete for s in [poem[i:i+3] for i in range(0, len(poem), 3) if len(poem[i:i+3]) == 3]]

    # Model generalist
    gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
    gpt2_model.eval()
    if torch.cuda.is_available():
        gpt2_model.to("cuda")

    # Model poetic
    poet_tokenizer = AutoTokenizer.from_pretrained("ayazfau/GPT2-124M-poetry-RL")
    poet_model = AutoModelForCausalLM.from_pretrained("ayazfau/GPT2-124M-poetry-RL")
    poet_model.eval()
    if torch.cuda.is_available():
        poet_model.to("cuda")


In [11]:
def analiza_calitativa_demo(rezultate_gpt2, rezultate_poet):
    import matplotlib.pyplot as plt
    import pandas as pd

    df = pd.DataFrame(rezultate_gpt2 + rezultate_poet)

    # Diferente scor BLEU
    scoruri_c1 = df.groupby("model")["bleu"].mean().reset_index()
    print("\nScor BLEU mediu pentru fiecare model:")
    for _, row in scoruri_c1.iterrows():
        print(f"→ {row['model']:<12s} | BLEU: {row['bleu']:.4f}")

    plt.figure()
    plt.bar(scoruri_c1['model'], scoruri_c1['bleu'], color=['purple', 'pink'])
    plt.title(" Comparatie BLEU mediu")
    plt.ylabel("BLEU")
    plt.grid(axis='y')
    plt.show()

    # Daca versurile din prompt sunt in limba engleza, romana, romana si corpusul de antrenare este in limba engleza
    prompts = {
        "EN prompt": "Is this a time to be cloudy and sad,",
        "RO prompt": "Te uita cum ninge decembre...",
        "RO prompt pe model EN": "Te uita cum ninge decembre..."
    }

    for label, prompt in prompts.items():
        print(f"\n {label}")
        
        inp_gpt2 = gpt2_tokenizer.encode(prompt + "\n", return_tensors="pt").to(gpt2_model.device)
        out_gpt2 = gpt2_model.generate(inp_gpt2, max_length=inp_gpt2.shape[1]+50, do_sample=True, temperature=0.7, top_p=0.9, pad_token_id=gpt2_tokenizer.eos_token_id)
        rezultat_gpt2 = gpt2_tokenizer.decode(out_gpt2[0], skip_special_tokens=True).replace('\n', ' ')
        print(f"→ GPT-2: {rezultat_gpt2.strip()}")

        inp_poet = poet_tokenizer.encode(prompt + "\n", return_tensors="pt").to(poet_model.device)
        out_poet = poet_model.generate(inp_poet, max_length=inp_poet.shape[1]+50, do_sample=True, temperature=0.7, top_p=0.9, pad_token_id=poet_tokenizer.eos_token_id)
        rezultat_poet = poet_tokenizer.decode(out_poet[0], skip_special_tokens=True).replace('\n', ' ')
        print(f"→ Poetic-GPT2: {rezultat_poet.strip()}")

    # Personalizare stil pastel prin prompting
    pastel_prompt = "In the light of sunset, the field blooms in silence"
    styled_prompt = pastel_prompt + "\n[style: pastel nature imagery]"
    inp_pastel = poet_tokenizer.encode(styled_prompt, return_tensors="pt").to(poet_model.device)
    out_pastel = poet_model.generate(inp_pastel, max_length=inp_pastel.shape[1]+50, do_sample=True, temperature=0.6, top_p=0.85, pad_token_id=poet_tokenizer.eos_token_id)
    rezultat = poet_tokenizer.decode(out_pastel[0], skip_special_tokens=True).replace('\n', ' ')
    print(f"\n Stil pastel: {rezultat.strip()}")


In [ ]:
def rulare_lab():
    init_model_si_date()
    parametri = [(50, 0.6, 0.85), (60, 0.7, 0.9), (50, 0.8, 0.95)]

    # CERINTA a 
    print("\n Evaluare model generalist GPT2")
    rezultate_gpt2 = evalueaza_model(gpt2_model, gpt2_tokenizer, strofe, parametri, "GPT2", num_strofe=5)

    # CERINTA b
    print("\n Evaluare model poetic specializat")
    rezultate_poet = evalueaza_model(poet_model, poet_tokenizer, strofe, parametri, "Poetic-GPT2", num_strofe=5)

    # Afisare scoruri BLEU comparativ
    df_rezultate = pd.DataFrame(rezultate_gpt2 + rezultate_poet)
    avg_bleu = df_rezultate.groupby("model")["bleu"].mean().reset_index()
    print("\n Scoruri BLEU medii pe model:")
    for _, row in avg_bleu.iterrows():
        print(f"→ {row['model']:12s}: {row['bleu']:.4f}")

    # Grafic BLEU
    plt.figure(figsize=(6, 4))
    plt.bar(avg_bleu['model'], avg_bleu['bleu'], color=['purple', 'pink'])
    plt.title("Scor BLEU mediu pe model")
    plt.ylabel("BLEU")
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()
    
    # CERINTA c
    try:
        rezultate_gpt2 = evalueaza_model(gpt2_model, gpt2_tokenizer, strofe, [(60, 0.7, 0.9)], "GPT2")
        rezultate_poet = evalueaza_model(poet_model, poet_tokenizer, strofe, [(60, 0.7, 0.9)], "Poetic-GPT2")
        analiza_calitativa_demo(rezultate_gpt2, rezultate_poet)

        # Salvam poezia cu BLEU maxim
        ofera_max = max(rezultate_poet, key=lambda x: x['bleu'])
        with open("poezie_preferata.txt", "w", encoding="utf-8") as f:
            f.write("Prompt:\n" + ofera_max['prompt'] + "\n\nGenerat:\n" + "\n".join(ofera_max['generat']))
        print("Poezia a fost salvata in 'poezie_preferata.txt'")
    except Exception as e:
        print("Eroare la generare demo analiza c:", e)
rulare_lab()


# CERINTA c.1: Diferentele de calitate intre GPT2 generalist si modelul poetic.
# Modelul poetic produce versuri mai stilizate si apropiate de stilul poetic,
# avand scoruri BLEU mai bune comparativ cu modelul generalist.

# CERINTA c.2: Prompt in limba engleza.
# Atat modelul generalist, cat si cel poetic genereaza texte coerente si fluente
# atunci cand promptul este in limba engleza, limbajul lor principala.

# CERINTA c.3: Prompt in limba romana.
# Modelele au dificultati in a genera texte corecte in romana,
# deoarece nu sunt antrenate pe acest limbaj, rezultand texte incoerente.

# CERINTA c.4: Prompt in romana, model antrenat pe engleza.
# Cand promptul este in romana dar modelele sunt antrenate in engleza,
# generarea ramane preponderent in engleza, afectand calitatea textului.

# CERINTA c.5: Personalizarea stilului pastel.
# Stilul poetic poate fi directionat folosind prompt engineering,
# adaugand indicatii stilistice in prompt pentru a genera imagini tematice.